Below will be code that reads in data from the serial (from the Arduino Uno) to be read and manipulated in Thonny's terminal

The code in the ArduinoIDE reading in the position of the accelerometer and gyroscope. 
N.B. This was written using ClaudeAI since I am not proficient in C++ and needed a fast way to test the code (before I dedicate time to learning it).

In [ ]:
#include <Wire.h>

// MPU6050 I2C address
#define MPU6050_ADDR 0x68

// Raw sensor data
int16_t accelX, accelY, accelZ;

// Converted acceleration values (in g's)
float ax, ay, az;

// Tilt angles (in degrees)
float tiltX, tiltY;

void setup() {
  Serial.begin(9600);
  Wire.begin();
  
  Serial.println("Initializing MPU6050...");
  
  // Initialize MPU6050
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(0x6B);  // PWR_MGMT_1 register
  Wire.write(0);     // Wake up the MPU6050
  Wire.endTransmission(true);
  
  // Optional: Set accelerometer range to ±2g (default)
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(0x1C);  // ACCEL_CONFIG register
  Wire.write(0x00);  // ±2g range
  Wire.endTransmission(true);
  
  Serial.println("MPU6050 initialized successfully!");
  Serial.println();
  Serial.println("Acceleration (g) | Tilt (degrees)");
  Serial.println("AX\tAY\tAZ\t| TiltX\tTiltY");
  Serial.println("----------------------------------------");
  
  delay(100);
}

void loop() {
  // Read accelerometer data
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(0x3B);  // Starting register for accelerometer data
  Wire.endTransmission(false);
  Wire.requestFrom(MPU6050_ADDR, 6, true);
  
  // Check if we received the expected amount of data
  if (Wire.available() == 6) {
    // Read 6 bytes (2 bytes per axis)
    accelX = (Wire.read() << 8 | Wire.read());
    accelY = (Wire.read() << 8 | Wire.read());
    accelZ = (Wire.read() << 8 | Wire.read());
    
    // Convert raw values to g's (±2g range, 16-bit resolution)
    ax = accelX / 16384.0;
    ay = accelY / 16384.0;
    az = accelZ / 16384.0;
    
    // Calculate tilt angles using accelerometer data
    // Tilt around X-axis (pitch) - forward/backward tilt
    tiltX = atan2(ay, sqrt(ax * ax + az * az)) * 180.0 / PI;
    
    // Tilt around Y-axis (roll) - left/right tilt
    tiltY = atan2(-ax, sqrt(ay * ay + az * az)) * 180.0 / PI;
    
    // Print acceleration vector (3 decimal places)
    Serial.print(ax, 3);
    Serial.print("\t");
    Serial.print(ay, 3);
    Serial.print("\t");
    Serial.print(az, 3);
    Serial.print("\t| ");
    
    // Print tilt angles (1 decimal place)
    Serial.print(tiltX, 1);
    Serial.print("\t");
    Serial.println(tiltY, 1);
  } else {
    Serial.println("Error reading from MPU6050!");
  }
  
  delay(100);  // Update every 100ms
}

This is the code in Python to read this data -- based on the context of this tutorial: 
https://www.youtube.com/watch?v=AHr94RtMj1A&t=39s

In [ ]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()

portList = []

for port in ports:
    portList.append(str(port))
    print(str(port))
    
val = input("Select port: COM")

for x in range(len(portList)):
    if portList[x].startswith("COM" + str(val)):
        portVar = "COM" + str(val)
        print(portList[x])
        
serialInst.baudrate = 9600
serialInst.port = portVar
serialInst.open()

while True:
    if serialInst.in_waiting: # if there is data in the buffer
        packet = serialInst.readline()
        decoded = packet.decode('utf')
        print(decoded)
        
# decode this data further so that it can be used with matplotlib
# make it easier to stop and start the flow of data through combinations of key presses (for example)

